## Building A Chatbot
We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This will cover the basics which will be helpful for those two more advanced topics.

In [55]:
import os
from dotenv import load_dotenv
load_dotenv(r"C:\Projects\.env")
from langchain_groq import ChatGroq
from langchain_core.messages import AIMessage, HumanMessage , SystemMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [56]:
groq_api_key = os.getenv("GROQ_API_KEY")

In [57]:
llm = ChatGroq(model="llama-3.3-70b-versatile",groq_api_key=groq_api_key)
response = llm.invoke("I am Soumo working as a product analyst in source desk")


In [58]:
response

AIMessage(content='Hello Soumo, nice to meet you. It sounds like you have an interesting role as a Product Analyst at Source Desk. What specific aspects of product analysis do you focus on, and what are some of the most challenging or rewarding parts of your job?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 47, 'total_tokens': 99, 'completion_time': 0.189090909, 'prompt_time': 0.002107768, 'queue_time': 0.057032799999999995, 'total_time': 0.191198677}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'finish_reason': 'stop', 'logprobs': None}, id='run--baf010a7-4b28-4cb5-8ff2-074cb161b34e-0', usage_metadata={'input_tokens': 47, 'output_tokens': 52, 'total_tokens': 99})

In [59]:
llm.invoke(    [
        HumanMessage(content="I am Soumo working as a product analyst in source desk"),
        AIMessage(content="Hello Soumo, it's nice to meet you. As a product analyst at Source Desk, you must be working on analyzing data to inform product decisions and drive business growth. What specific areas of product analysis do you focus on, such as customer behavior, market trends, or product optimization? I'm here to help and chat about your work if you'd like."),
        HumanMessage(content="What is My Name ?")
    ]
).content

'Your name is Soumo, and you work as a product analyst at Source Desk.'

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!


In [63]:
store = {}
def get_session_history(session_id: str)-> BaseChatMessageHistory: ## functio will be created to get the session history and 
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]
config1 = {"configurable": {"session_id":"s1"}}
config2 = {"configurable": {"session_id":"s2"}}
with_message_history = RunnableWithMessageHistory(llm,get_session_history)

In [66]:
config1 = {"configurable": {"session_id":"s1"}}
with_message_history.invoke(

    [HumanMessage(content="I am Soumo working as a product analyst in source desk"),
     AIMessage(content="Hello Soumo, it's nice to meet you. As a product analyst at Source Desk, you must be working on analyzing data to inform product decisions and drive business growth. What specific areas of product analysis do you focus on, such as customer behavior, market trends, or product optimization? I'm here to help and chat about your work if you'd like."),
     AIMessage(content=" \n\nAlso, I can offer some general information about product analysis if you're interested. Or we could discuss topics like data analysis, product development, or industry trends. Just let me know what's on your mind!"),
     HumanMessage(content="What is My Name ?"),
     AIMessage(content="Your name is Soumo, and you work as a product analyst at Source Desk."),
     HumanMessage(content="Where is SourceDesk located ?")],
    config=config1
).content

"You didn't mention the location of Source Desk, so I don't have that information. As a product analyst at Source Desk, you might know the location, but I don't have any details about it. Can you tell me where Source Desk is located?"

In [104]:
list(store.keys())

['s1', 's2']

In [107]:
config2 = {"configurable": {"session_id":"s2"}}
with_message_history.invoke(

    [
     HumanMessage(content="I am krish working as a product analyst in source desk"),
     AIMessage(content="Hello Krish, nice to meet you. It sounds like you have an interesting role as a Product Analyst at Source Desk. What specific aspects of product analysis do you focus on, and what are some of the most challenging or rewarding parts of your job?"),
     HumanMessage(content="Source desk is an IT firm, located in kolkata"),
     AIMessage(content="Hello Krish, nice to meet you. So, you work as a Product Analyst at Source Desk, which is an IT firm. That's a great industry to be in, with always evolving technologies and innovations.\n\nAs a Product Analyst, I assume you work closely with cross-functional teams to analyze data, identify trends, and inform product decisions. What kind of products or services does Source Desk offer, and how do you contribute to their development and growth?"),
     HumanMessage(content="whtat is source desk"),
     AIMessage(content="Unfortunately, I don't have any specific information about Source Desk, as it seems to be a private company. However, based on our conversation, I can make some general assumptions:\n\n* Source Desk is an IT firm, which means it likely provides information technology-related services or products.\n* It is located in Kolkata, India, which suggests it may be a regional or national company with a presence in Eastern India.\n* As a Product Analyst, you likely work on analyzing data, identifying trends, and informing product decisions to help drive the company's growth and success.\n\nIf you'd like to share more about Source Desk, I'd be happy to learn more! What kind of products or services does the company offer, and what's your role like as a Product Analyst?"),
     HumanMessage(content="Sggest me a proper road map to Swith my profile to data anlyst")
    ],
    config=config2
).content

"As a Product Analyst, you already have a strong foundation in analysis and problem-solving. To switch to a Data Analyst role, you'll need to build on your existing skills and acquire some new ones. Here's a suggested roadmap to help you make the transition:\n\n**Short-term goals (3-6 months)**\n\n1. **Brush up on statistical concepts**:\n\t* Review probability, statistics, and data modeling concepts.\n\t* Study resources: Khan Academy, Coursera, edX, or DataCamp.\n2. **Learn data visualization tools**:\n\t* Familiarize yourself with Tableau, Power BI, or D3.js.\n\t* Practice creating interactive dashboards and visualizations.\n3. **Improve your SQL skills**:\n\t* Learn advanced SQL concepts, such as subqueries, joins, and aggregations.\n\t* Practice writing efficient queries and optimizing database performance.\n4. **Get familiar with data manipulation and analysis libraries**:\n\t* Learn Python libraries like Pandas, NumPy, and Matplotlib.\n\t* Practice data cleaning, filtering, and 

In [108]:
store.keys()

dict_keys(['s1', 's2'])

In [109]:
for j in list(store.keys()):
    print(j)
    for i, mes in enumerate(store.get(j).messages):
        print("Type:", mes.type, "Content:", mes.content)

s1
Type: human Content: I am Soumo working as a product analyst in source desk
Type: ai Content: Hello Soumo, it's nice to meet you. As a product analyst at Source Desk, you must be working on analyzing data to inform product decisions and drive business growth. What specific areas of product analysis do you focus on, such as customer behavior, market trends, or product optimization? I'm here to help and chat about your work if you'd like.
Type: ai Content:  

Also, I can offer some general information about product analysis if you're interested. Or we could discuss topics like data analysis, product development, or industry trends. Just let me know what's on your mind!
Type: human Content: What is My Name ?
Type: ai Content: Your name is Soumo, and you work as a product analyst at Source Desk.
Type: human Content: I am Soumo working as a product analyst in source desk
Type: ai Content: Hello Soumo, it's nice to meet you. As a product analyst at Source Desk, you must be working on anal

In [110]:
from IPython.display import Markdown, display

for j in list(store.keys()):
    md = f"### Session: `{j}`\n"
    for i, mes in enumerate(store.get(j).messages, 1):
        md += f"**{i}. {mes.type.capitalize()}**: {mes.content}\n\n"
    display(Markdown(md))

### Session: `s1`
**1. Human**: I am Soumo working as a product analyst in source desk

**2. Ai**: Hello Soumo, it's nice to meet you. As a product analyst at Source Desk, you must be working on analyzing data to inform product decisions and drive business growth. What specific areas of product analysis do you focus on, such as customer behavior, market trends, or product optimization? I'm here to help and chat about your work if you'd like.

**3. Ai**:  

Also, I can offer some general information about product analysis if you're interested. Or we could discuss topics like data analysis, product development, or industry trends. Just let me know what's on your mind!

**4. Human**: What is My Name ?

**5. Ai**: Your name is Soumo, and you work as a product analyst at Source Desk.

**6. Human**: I am Soumo working as a product analyst in source desk

**7. Ai**: Hello Soumo, it's nice to meet you. As a product analyst at Source Desk, you must be working on analyzing data to inform product decisions and drive business growth. What specific areas of product analysis do you focus on, such as customer behavior, market trends, or product optimization? I'm here to help and chat about your work if you'd like.

**8. Ai**:  

Also, I can offer some general information about product analysis if you're interested. Or we could discuss topics like data analysis, product development, or industry trends. Just let me know what's on your mind!

**9. Human**: What is My Name ?

**10. Ai**: Your name is Soumo, and you work as a product analyst at Source Desk.

**11. Human**: Where is SourceDesk located ?

**12. Ai**: You didn't mention the location of Source Desk, so I don't have that information. As a product analyst at Source Desk, you might know the location, but I don't have any details about it. Can you tell me where Source Desk is located?



### Session: `s2`
**1. Human**: I am krish working as a product analyst in source desk

**2. Ai**: Hello Krish, nice to meet you. It sounds like you have an interesting role as a Product Analyst at Source Desk. What specific aspects of product analysis do you focus on, and what are some of the most challenging or rewarding parts of your job?

**3. Human**: I am krish working as a product analyst in source desk

**4. Ai**: Hello Krish, nice to meet you. It sounds like you have an interesting role as a Product Analyst at Source Desk. What specific aspects of product analysis do you focus on, and what are some of the most challenging or rewarding parts of your job?

**5. Human**: Source desk is an IT firm

**6. Ai**: Hello Krish, nice to meet you. So, you work as a Product Analyst at Source Desk, which is an IT firm. That's a great industry to be in, with always evolving technologies and innovations.

As a Product Analyst, I assume you work closely with cross-functional teams to analyze data, identify trends, and inform product decisions. What kind of products or services does Source Desk offer, and how do you contribute to their development and growth?

**7. Human**: I am krish working as a product analyst in source desk

**8. Ai**: Hello Krish, nice to meet you. It sounds like you have an interesting role as a Product Analyst at Source Desk. What specific aspects of product analysis do you focus on, and what are some of the most challenging or rewarding parts of your job?

**9. Human**: Source desk is an IT firm, located in kolkata

**10. Ai**: Hello Krish, nice to meet you. So, you work as a Product Analyst at Source Desk, which is an IT firm. That's a great industry to be in, with always evolving technologies and innovations.

As a Product Analyst, I assume you work closely with cross-functional teams to analyze data, identify trends, and inform product decisions. What kind of products or services does Source Desk offer, and how do you contribute to their development and growth?

**11. Human**: whtat is source desk

**12. Ai**: Unfortunately, I don't have any specific information about Source Desk, as it seems to be a private company. However, based on our conversation, I can make some general assumptions:

* Source Desk is an IT firm, which means it likely provides information technology-related services or products.
* It is located in Kolkata, India, which suggests it may be a regional or national company with a presence in Eastern India.
* As a Product Analyst, you likely work on analyzing data, identifying trends, and informing product decisions to help drive the company's growth and success.

If you'd like to share more about Source Desk, I'd be happy to learn more! What kind of products or services does the company offer, and what's your role like as a Product Analyst?

**13. Human**: I am krish working as a product analyst in source desk

**14. Ai**: Hello Krish, nice to meet you. It sounds like you have an interesting role as a Product Analyst at Source Desk. What specific aspects of product analysis do you focus on, and what are some of the most challenging or rewarding parts of your job?

**15. Human**: Source desk is an IT firm, located in kolkata

**16. Ai**: Hello Krish, nice to meet you. So, you work as a Product Analyst at Source Desk, which is an IT firm. That's a great industry to be in, with always evolving technologies and innovations.

As a Product Analyst, I assume you work closely with cross-functional teams to analyze data, identify trends, and inform product decisions. What kind of products or services does Source Desk offer, and how do you contribute to their development and growth?

**17. Human**: whtat is source desk

**18. Ai**: Unfortunately, I don't have any specific information about Source Desk, as it seems to be a private company. However, based on our conversation, I can make some general assumptions:

* Source Desk is an IT firm, which means it likely provides information technology-related services or products.
* It is located in Kolkata, India, which suggests it may be a regional or national company with a presence in Eastern India.
* As a Product Analyst, you likely work on analyzing data, identifying trends, and informing product decisions to help drive the company's growth and success.

If you'd like to share more about Source Desk, I'd be happy to learn more! What kind of products or services does the company offer, and what's your role like as a Product Analyst?

**19. Human**: Sggest me a proper road map to Swith my profile to data anlyst

**20. Ai**: As a Product Analyst, you already have a strong foundation in analysis and problem-solving. To switch to a Data Analyst role, you'll need to build on your existing skills and acquire some new ones. Here's a suggested roadmap to help you make the transition:

**Short-term goals (3-6 months)**

1. **Brush up on statistical concepts**:
	* Review probability, statistics, and data modeling concepts.
	* Study resources: Khan Academy, Coursera, edX, or DataCamp.
2. **Learn data visualization tools**:
	* Familiarize yourself with Tableau, Power BI, or D3.js.
	* Practice creating interactive dashboards and visualizations.
3. **Improve your SQL skills**:
	* Learn advanced SQL concepts, such as subqueries, joins, and aggregations.
	* Practice writing efficient queries and optimizing database performance.
4. **Get familiar with data manipulation and analysis libraries**:
	* Learn Python libraries like Pandas, NumPy, and Matplotlib.
	* Practice data cleaning, filtering, and grouping using these libraries.

**Mid-term goals (6-12 months)**

1. **Learn machine learning fundamentals**:
	* Study supervised and unsupervised learning concepts.
	* Learn popular algorithms like linear regression, decision trees, and clustering.
2. **Explore data science tools and technologies**:
	* Learn about big data technologies like Hadoop, Spark, or NoSQL databases.
	* Familiarize yourself with cloud-based data platforms like AWS, GCP, or Azure.
3. **Develop data storytelling skills**:
	* Learn to communicate complex data insights effectively.
	* Practice creating reports, presentations, and dashboards that tell a story.
4. **Participate in data analysis projects**:
	* Join online communities like Kaggle, Reddit (r/dataanalysis), or Data Science subreddit.
	* Participate in hackathons or data science competitions to practice your skills.

**Long-term goals (1-2 years)**

1. **Pursue a certification in data analysis**:
	* Consider certifications like Google Data Analytics, Microsoft Certified: Data Analyst, or Certified Data Analyst (CDA).
2. **Learn advanced data analysis techniques**:
	* Study topics like time series analysis, forecasting, or text analysis.
	* Learn to work with unstructured data, such as images, audio, or text.
3. **Stay up-to-date with industry trends and developments**:
	* Attend conferences, meetups, or webinars to learn about new tools and technologies.
	* Network with professionals in the field to stay informed about best practices.
4. **Consider a graduate degree in data science**:
	* If you're interested in advanced research or leadership roles, a master's degree in data science might be beneficial.

**Additional tips**

1. **Build a strong portfolio**: Showcase your data analysis projects and accomplishments on platforms like GitHub or Kaggle.
2. **Network with professionals**: Attend industry events, join online communities, and connect with data analysts on LinkedIn.
3. **Stay curious and keep learning**: The field of data analysis is constantly evolving, so it's essential to stay updated with new tools, technologies, and techniques.

Remember, switching to a Data Analyst role takes time, effort, and dedication. Focus on building your skills, networking, and gaining practical experience to increase your chances of success. Good luck!



In [112]:
store.keys()

dict_keys(['s1', 's2'])